In [137]:
#Importing Libraries and Reading Data
import numpy as np
import pandas as pd
data = pd.read_csv('data.csv')
data

,Text,Category
0,"In 2018, D'Ernest Johnson wasn't signed by any...",Sports
1,Tom Brady congratulated Aaron Rodgers for beco...,Sports
2,Leading Formula One drivers defended the popul...,Sports
3,Greece's Maria Sakkari will make her debut at ...,Sports
4,"Gymnast Simone Biles, who disclosed her mental...",Sports
5,The NBA is facing another incident involving C...,Sports
6,An Italian DJ claims he suffered a number of i...,Sports
7,Canadian athletes looking to compete for Team ...,Sports
8,It's not often an NBA star can quietly make hi...,Sports
9,With 100 race wins and seven world championshi...,Sports


In [138]:
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))

<h1>Creating Text Classifier Class

In [139]:
class TextClassifier():
    def __init__(self, dataset):
        #Splitting the given data
        txt_train, txt_test, cat_train, cat_test = train_test_split(dataset.Text, dataset.Category, test_size=.2)
        #Creating training set and test set for class
        self.training_set = pd.DataFrame({'Text':txt_train,'Category':cat_train})
        self.test_set = pd.DataFrame({'Text':txt_test,'Category':cat_test})
        print('Model Created...')

    def stopwords_and_stemming(self, text_data):
        for text in text_data:
            text = text.split()
            clean_data = []
            for word in text:
                if word not in stop_words:
                    clean_data.append(stemmer.stem(word))
            return clean_data

    def print_training_set(self):
        print(self.training_set)